In [23]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib_venn import venn2, venn3

In [26]:
out_dir = Path("/rsrch4/home/mol_cgenesis/EMC_BIC_rsrch4/nkdang/Splicing_Projects/results/viz/venn")
out_dir.mkdir(parents=True, exist_ok=True)

def draw_venn(set1, set2, set3=None, labels=tuple(['set1','set2','set3']), title='Untitle', out_path=Path.cwd()):
    out_path.mkdir(parents=True, exist_ok=True)
    if set3:
        if len(labels)!=3:
            return "Wrong number of labels"
        else:
            intersect12 = set(set1).intersection(set(set2))
            intersect23 = set(set2).intersection(set(set3))
            intersect13 = set(set1).intersection(set(set3))
            intersect123 = set(set1).intersection(set(set2)).intersection(set(set3))
            venn3(subsets={'100': len(set1)-len(intersect12)-len(intersect13)+len(intersect123), 
                           '010': len(set2)-len(intersect12)-len(intersect23)+len(intersect123), 
                           '110': len(intersect12)-len(intersect123),
                           '001': len(set3)-len(intersect13)-len(intersect23)+len(intersect123), 
                           '101': len(intersect13)-len(intersect123), 
                           '011': len(intersect23)-len(intersect123),
                           '111': len(intersect123)}, set_labels=labels)
            plt.title(title)
            plt.savefig(out_path.joinpath("fig.png"), dpi=300)
            plt.close()
            with open(out_path.joinpath(f"sharedGenes_{labels[0]}_{labels[1]}.txt"), 'w') as fp:
                fp.writelines([i+"\n" for i in intersect12])
            with open(out_path.joinpath(f"sharedGenes_{labels[0]}_{labels[2]}.txt"), 'w') as fp:
                fp.writelines([i+"\n" for i in intersect13])
            with open(out_path.joinpath(f"sharedGenes_{labels[1]}_{labels[2]}.txt"), 'w') as fp:
                fp.writelines([i+"\n" for i in intersect23])
            with open(out_path.joinpath(f"sharedGenes_{labels[0]}_{labels[1]}_{labels[2]}.txt"), 'w') as fp:
                fp.writelines([i+"\n" for i in intersect123])
    else:
        if len(labels)!=2:
            return "Wrong number of labels"
        else:
            intersect12 = set(set1).intersection(set(set2))
            venn2(subsets={'10': len(set1)-len(intersect12), 
                           '01': len(set2)-len(intersect12), 
                           '11': len(intersect12)}, set_labels=labels)
            plt.title(title)
            plt.savefig(out_path.joinpath("fig.png"), dpi=300)
            plt.close()
            with open(out_path.joinpath(f"sharedGenes_{labels[0]}_{labels[1]}.txt"), 'w') as fp:
                fp.writelines([i+"\n" for i in intersect12])

In [32]:
for mut in ['W377A','Y112A']:
    df1 = pd.read_csv(f"/rsrch4/home/mol_cgenesis/EMC_BIC_rsrch4/nkdang/Splicing_Projects/results/new/DE_featureCounts/Yalong/EV_WT/EV_WT/Yalong_Hela_EV_WT_sig.csv", index_col=0)
    df2 = pd.read_csv(f"/rsrch4/home/mol_cgenesis/EMC_BIC_rsrch4/nkdang/Splicing_Projects/results/new/DE_featureCounts/Yalong/EV_WT/{mut}_EV/Yalong_Hela_{mut}_EV_sig.csv", index_col=0)
    df3 = pd.read_csv(f"/rsrch4/home/mol_cgenesis/EMC_BIC_rsrch4/nkdang/Splicing_Projects/results/new/DE_featureCounts/Yalong/EV_WT/{mut}_WT/Yalong_Hela_{mut}_WT_sig.csv", index_col=0)

    temp1 = [i for i in df1[df1['log2FoldChange']>0].index]
    temp2 = [i for i in df2[df2['log2FoldChange']>0].index]
    temp3 = [i for i in df3[df3['log2FoldChange']>0].index]
    draw_venn(temp1,temp2,labels=tuple(["EV_WT",f"{mut}_EV"]), title="Venn - up regulated significant genes", out_path=out_dir.joinpath("Yalong").joinpath(f"{mut}_WT").joinpath(f"EV_WT-{mut}_EV_up"))
    draw_venn(temp2,temp3,labels=tuple([f"{mut}_EV",f"{mut}_WT"]), title="Venn - up regulated significant genes", out_path=out_dir.joinpath("Yalong").joinpath(f"{mut}_WT").joinpath(f"{mut}_EV-{mut}_WT_up"))
    draw_venn(temp1,temp3,labels=tuple(["EV_WT",f"{mut}_WT"]), title="Venn - up regulated significant genes", out_path=out_dir.joinpath("Yalong").joinpath(f"{mut}_WT").joinpath(f"EV_WT-{mut}_WT_up"))
    draw_venn(temp1,temp2,temp3,labels=tuple(["EV_WT",f"{mut}_EV",f"{mut}_WT"]), title="Venn - up regulated significant genes", out_path=out_dir.joinpath("Yalong").joinpath(f"{mut}_WT").joinpath(f"EV_WT-{mut}_EV-{mut}_WT_up"))

    temp1 = [i for i in df1[df1['log2FoldChange']<0].index]
    temp2 = [i for i in df2[df2['log2FoldChange']<0].index]
    temp3 = [i for i in df3[df3['log2FoldChange']<0].index]
    draw_venn(temp1,temp2,labels=tuple(["EV_WT",f"{mut}_EV"]), title="Venn - down regulated significant genes", out_path=out_dir.joinpath("Yalong").joinpath(f"{mut}_WT").joinpath(f"EV_WT-{mut}_EV_down"))
    draw_venn(temp2,temp3,labels=tuple([f"{mut}_EV",f"{mut}_WT"]), title="Venn - down regulated significant genes", out_path=out_dir.joinpath("Yalong").joinpath(f"{mut}_WT").joinpath(f"{mut}_EV-{mut}_WT_down"))
    draw_venn(temp1,temp3,labels=tuple(["EV_WT",f"{mut}_WT"]), title="Venn - down regulated significant genes", out_path=out_dir.joinpath("Yalong").joinpath(f"{mut}_WT").joinpath(f"EV_WT-{mut}_WT_down"))
    draw_venn(temp1,temp2,temp3,labels=tuple(["EV_WT",f"{mut}_EV",f"{mut}_WT"]), title="Venn - down regulated significant genes", out_path=out_dir.joinpath("Yalong").joinpath(f"{mut}_WT").joinpath(f"EV_WT-{mut}_EV-{mut}_WT_down"))